In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA
import scipy

In [2]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)
# path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/liverpool_liver_31_test/"

In [3]:
all_AAs=['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p','q', 'r', 's', 't', 'v', 'w', 'y']
def charcount(pepSeq):
    res={key: 0 for key in all_AAs}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [4]:
def getPeptiedsInfo(protien,time):
    print(protien,time)
    peptides=[x.split(f'_{time}_')[0] for x in os.listdir(path) if (protien+".csv" in x)]
    peptides=[x  for x in peptides if 'A' in x]
    peptidesinfo=dict()
    print(len(peptides))
    

    for k in range(len(peptides)):
        
        data=pd.read_csv(f"{path}{peptides[k]}_{time}_{protien}.csv")
        data["pxt_neh"]=data.NEH*data.pxt
        data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
        minval=min(data.RMSE)

        data=data[data.RMSE==minval]

        peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                               "charcount":charcount(peptides[k])}
           
    visited=set()
    visited.add(peptides[0].split('_')[1])
    coefficients=[]
    for k in range(0,len(peptides)-1):
        if(peptides[k+1].split('_')[1] in visited):
            continue

        pep1=peptidesinfo[peptides[k]]
        pep2=peptidesinfo[peptides[k+1]]    
        visited.add(peptides[k].split('_')[1])

        r=pep1["pxt_neh"]/pep2["pxt_neh"]
        coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
        
    return peptides,peptidesinfo,all_AAs,np.array(coefficients)


In [5]:
def compute_result(peptides,peptidesinfo,all_AAs,sol):
    t_neh,e_neh=[],[]
    for i in range (len(peptides)):
        pep1=peptidesinfo[peptides[i]]
        p_neh=0
        for index,c in enumerate(all_AAs): 
            p_neh+=(pep1["charcount"][c]*sol[index]) 
        t_neh.append(pep1["T_NEH"])
        e_neh.append(p_neh)

    res=pd.DataFrame([peptides,t_neh,e_neh])
    res=res.transpose()
    res.columns=["peptides","True_NEH","e_NEH"]
    res["dif"]=abs(res.True_NEH-res.e_NEH)
#     print(np.mean(res.dif),np.median(res.dif),np.std(res.dif))
    return res.sort_values(by='dif')

In [6]:
time=31
a_val=4
w_val=0
# proteins=["CPSM_MOUSE","FAS_MOUSE","DHB4_MOUSE","ASSY_MOUSE","ATPA_MOUSE","ATPB_MOUSE","BHMT1_MOUSE",
#          "CLH1_MOUSE","DHE3_MOUSE","DHSO_MOUSE","ECHA_MOUSE",
#          "G3P_MOUSE","HMCS2_MOUSE","HYES_MOUSE","MYH9_MOUSE","PYC_MOUSE","PYGL_MOUSE","SAHH_MOUSE",
#          "SARDH_MOUSE","SBP1_MOUSE","THIM_MOUSE","TKT_MOUSE"]
# proteins=pd.read_csv("H:/Warehouse/Data/liverpool/liver/6_23_2022_c/"+"Analyzed_Proteins_and_Their_Rates.csv").Proteins.unique()

path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/" 
proteins=np.unique(["_".join(x.split('_')[4:]).replace('.csv','') for x in os.listdir(path) if ("_MOUSE.csv" in x)])

coefficients=np.array([])
_peptides,_peptidesinfo=[],[]
for protine in proteins:
    try:
        peptides1,peptidesinfo1,all_AAs1,coefficients1=getPeptiedsInfo(protine,time) 
        print(protine,len(all_AAs1)==20)
        if(coefficients.shape[0]==0):
            coefficients=coefficients1
        else:
            coefficients=np.concatenate((coefficients1,coefficients), axis=0)
        _peptides.append(peptides1)
        _peptidesinfo.append(peptidesinfo1)
    except:
        continue
    
# sol=list(scipy.optimize.nnls(coefficients[:,2 :],-a_val*coefficients[:,0]-w_val*coefficients[:,1])[0])
sol=list(scipy.optimize.nnls(coefficients[:,1:],-a_val*coefficients[:,0])[0])

print("====>",len(sol))
sol.insert(0,a_val) 
# sol.insert(1,w_val) 
print("\nValues\n","\n".join([all_AAs1[index]+"= "+str(x) for index,x in enumerate(sol)]))
res=pd.DataFrame()
for i in range(len(proteins)):
    res=pd.concat([res,compute_result(_peptides[i],_peptidesinfo[i],all_AAs1,sol)])

print("\nstats",np.mean(res.dif),np.median(res.dif),np.std(res.dif))    
 

ACADM_MOUSE 31
7
ACADM_MOUSE True
ACON_MOUSE 31
8
ACON_MOUSE True
ACSM2_MOUSE 31
12
ACSM2_MOUSE True
ALBU_MOUSE 31
17
ALBU_MOUSE True
AMPL_MOUSE 31
8
AMPL_MOUSE True
ASSY_MOUSE 31
7
ASSY_MOUSE True
AT1A1_MOUSE 31
12
AT1A1_MOUSE True
ATPA_MOUSE 31
13
ATPA_MOUSE True
ATPB_MOUSE 31
16
ATPB_MOUSE True
CH60_MOUSE 31
9
CH60_MOUSE True
CLH1_MOUSE 31
17
CLH1_MOUSE True
F16P1_MOUSE 31
8
F16P1_MOUSE True
GRP75_MOUSE 31
6
GRP75_MOUSE True
MMSA_MOUSE 31
11
MMSA_MOUSE True
MYH9_MOUSE 31
8
MYH9_MOUSE True
PYC_MOUSE 31
14
PYC_MOUSE True
SPTB2_MOUSE 31
6
SPTB2_MOUSE True
SPTN1_MOUSE 31
10
SPTN1_MOUSE True
THIL_MOUSE 31
9
THIL_MOUSE True
====> 19

Values
 a= 4
c= 1.5890925641897424
d= 2.140981968310892
e= 3.6311994007533075
f= 0.07157539915823963
g= 1.5713231227129356
h= 1.9814701560613714
i= 0.6569441743949092
k= 0.0
l= 0.6086843700604827
m= 0.485155724881216
n= 0.8170629158282595
p= 1.6657679892072865
q= 3.358592518490666
r= 0.7466795867741833
s= 1.441863978115052
t= 0.04422828746504352
v= 0.47023547

In [7]:
print("\nValues\n","\n".join([all_AAs1[index]+"= "+str(x) for index,x in enumerate(sol)]))


Values
 a= 4
c= 1.5890925641897424
d= 2.140981968310892
e= 3.6311994007533075
f= 0.07157539915823963
g= 1.5713231227129356
h= 1.9814701560613714
i= 0.6569441743949092
k= 0.0
l= 0.6086843700604827
m= 0.485155724881216
n= 0.8170629158282595
p= 1.6657679892072865
q= 3.358592518490666
r= 0.7466795867741833
s= 1.441863978115052
t= 0.04422828746504352
v= 0.4702354765741533
w= 0.0
y= 0.0


In [8]:
print("proteins",len(proteins),"peptides",res.shape[0])

proteins 19 peptides 198


In [9]:
res=res.sort_values(by='dif')
res.head(20)

peptides True_NEH      e_NEH       dif
3           _QALQDTLALYK_2     20.0  18.728448  1.271552
7        _GVMLAVDAVIAELK_3     25.0   23.11368   1.88632
6        _GVMLAVDAVIAELK_2     25.0   23.11368   1.88632
0          _ALEHFTDLYDIK_3     18.0   15.88475   2.11525
15        _TcVADESAANcDK_2     28.0  25.864739  2.135261
8          _ENYGELADccTK_2     22.0  19.622865  2.377135
4        _LAALADQWQFLVQK_2     29.0  26.584624  2.415376
7           _NAEQFKDQADK_3     26.0  23.518987  2.481013
6           _NAEQFKDQADK_2     26.0  23.518987  2.481013
2           _FAELVYTGFWH_2     15.0  12.450292  2.549708
12         _VVDALGNAIDGK_2     21.0  18.447773  2.552227
9           _TLADAEGDVFR_2     22.0   19.42589   2.57411
2           _AEMDAAVEScK_2     28.0  25.389729  2.610271
3           _HALIIYDDLSK_2     17.0  14.236555  2.763445
0          _AQFEGIVTDLIK_2     20.0  17.210709  2.789291
2           _EAYPGDVFYLH_2     19.0  16.141238  2.858762
4          _LGTAAIQGAIEK_2     27.0  24.099239  2.900761
3   _IAALQAFADQLIAVDHYAK_3     43.0  40.053687  2.946313
0         _AVAGDASESALLK_2     31.0  27.914837  3.085163
10         _NLQNLLILTAIK_2     16.0  12.785572  3.214428

In [10]:
res.tail(20)

peptides True_NEH      e_NEH        dif
2   _AIAELGIYPAVDPLDSTSR_3     40.0  31.492151   8.507849
5     _ENGVDAVHPGYGFLSER_2     35.0  26.390925   8.609075
7   _VASNPYTWFTMEALEETWR_2     34.0  25.333309   8.666691
8   _VASNPYTWFTMEALEETWR_3     34.0  25.333309   8.666691
7         _VAMSHFEPSEYIR_3     29.0  20.223955   8.776045
15  _VLDSGAPIKIPVGPETLGR_3     34.0  25.187955   8.812045
2        _HQAFEAELSANQSR_3     42.0  33.088784   8.911216
4         _HSSLAGcQIINYR_3     28.0  18.870522   9.129478
6    _SSLSSAQADFNQLAELDR_2     45.0  35.859175   9.140825
7      _ENPTTFMGHYLHEVAR_4     31.0   21.75028    9.24972
6      _ENPTTFMGHYLHEVAR_3     31.0   21.75028    9.24972
0    _EVYMGNVIQGGEGQAPTR_2     39.0  29.621186   9.378814
1    _EVYMGNVIQGGEGQAPTR_3     39.0  29.621186   9.378814
5    _SAGVDDQENWHEGKENIR_4     42.0   32.60812    9.39188
0   _APVVMGSSEDVQEFLEIYR_2     40.0  30.493737   9.506263
1   _APVVMGSSEDVQEFLEIYR_3     40.0  30.493737   9.506263
12      _RPISADSAIMNPASK_3     35.0  25.160896   9.839104
5     _AISFVGSNQAGEYIFER_2     38.0  28.138383   9.861617
6     _AISFVGSNQAGEYIFER_3     38.0  28.138383   9.861617
8    _ISSVQSIVPALEIANAHR_3     43.0  32.046352  10.953648

In [11]:
temp=pd.DataFrame(coefficients)
temp.columns=all_AAs1
temp 

a         c         d         e         f         g         h  \
0   -1.099998  0.000000  0.000000 -0.099998 -2.099998  4.000000  0.000000   
1   -0.527779 -0.763889  0.000000  0.236111  1.000000 -0.763889 -0.763889   
2   -0.554052  1.000000  0.000000  0.148649  0.000000 -0.702702  1.000000   
3    1.178460 -0.910770  0.000000  1.000000  0.000000  0.178460  0.000000   
4   -0.080000  1.000000  0.000000  0.000000 -1.040000 -0.080000  0.000000   
..        ...       ...       ...       ...       ...       ...       ...   
164  0.206349  0.000000  0.000000  1.206349  1.000000  2.206349  0.000000   
165 -0.741935  0.000000  0.000000 -0.741935 -0.870968  0.129032 -0.870968   
166  1.022523  0.000000 -0.977477  2.000000  1.000000 -0.954954  1.000000   
167 -3.748663  0.000000  1.000000  0.000000  0.000000 -0.374332  0.000000   
168  0.021275  0.000000  0.000000  0.000000  0.000000  0.010637  0.000000   

            i         k         l         m         n         p         q  \
0   -5.299995 -2.099998  0.000000  1.000000  1.000000 -1.099998  2.000000   
1    2.236111  0.236111  0.000000 -0.763889 -1.527779  1.000000  0.000000   
2   -0.702702  0.148649 -0.851351  1.000000  2.000000  0.000000 -0.851351   
3    1.089230  0.089230 -0.821540  0.000000 -0.910770 -1.821540  0.089230   
4   -0.040000 -0.040000 -0.080000  0.000000  1.000000 -0.080000 -0.040000   
..        ...       ...       ...       ...       ...       ...       ...   
164 -1.380952  1.000000  1.000000  0.000000  1.000000  0.000000 -2.380952   
165  3.000000 -0.870968 -3.483871 -1.741935  0.000000  0.000000  2.129032   
166 -0.977477  0.022523  2.045046  1.022523 -0.977477 -0.977477 -1.932431   
167  1.000000  1.000000  0.812834  1.000000  1.000000 -0.187166  1.812834   
168  0.000000  0.000000  0.005319  0.000000  0.000000  0.005319  0.005319   

            r         s         t         v    w         y  
0    1.000000 -4.199997 -3.199997 -0.099998  0.0  1.000000  
1    0.000000  2.000000  1.236111  1.000000  0.0  0.000000  
2    0.000000  0.000000  0.148649  0.000000  0.0  0.000000  
3    0.000000 -0.910770 -2.643079 -0.910770  0.0  0.000000  
4    0.000000 -2.120000  1.920000  1.000000  0.0  0.000000  
..        ...       ...       ...       ...  ...       ...  
164 -0.793651  0.000000 -0.793651  1.000000  0.0 -1.587302  
165  1.000000 -0.870968  1.000000 -1.741935  0.0  2.000000  
166  0.000000  0.022523  0.000000  1.022523  0.0  0.000000  
167 -2.374332  1.000000 -2.374332 -1.374332  0.0  0.000000  
168  0.010637  0.000000  0.010637  0.010637  0.0  0.000000  

[169 rows x 20 columns]

In [12]:
# for c in range( len(coefficients)):
#     print(np.dot(coefficients[c],sol))